<h1>Powering Energy Storage Beyond Excel</h1>
<h2>This is a sample document to show the power of python over Excel files</h2>

In [ ]:
# Required Dependencies
import tabulate
import pandas as pd
from decimal import Decimal as D

In [ ]:
def get_data_for_a_stack(specifications, start, end):
    """
    Get specific data for a given Stack like
    1. Name
    2. List of Cycles
    3. Degradations list
    """
    stack_spec_df = specifications[start:end].fillna(0)
    name = stack_spec_df.iloc[0, 0]
    cycles = list(stack_spec_df.iloc[1].values[1:])
    stack_specs = stack_spec_df.iloc[2:, 1:51]

    return name, cycles, stack_specs


In [ ]:
def convert_data_to_df(file, columns, skiprows):
    """
    Read and excel file with specific columns and skip any non required 
    rows you encounter during reading rows.
    """
    specifications = pd.read_excel(file, skiprows=skiprows, usecols=columns)
    return specifications

<h3>Reading Data from excel to Store in Database for further use in other components</h3>

In [ ]:
# Rreading Data from Excel Sheet
total_specifications = convert_data_to_df("data/sample.xlsx", columns="AN:CL", skiprows=49)

"""
Reading data for a Stack to store it in the project for future use.
Stack Name = S230P CATL (CB2W0) (S230P 1.5+ hrs)
"""
stack_name, cycles, stack_spec_model_df = get_data_for_a_stack(total_specifications, start=0, end=23)

year = 0
print("Stack Name:", stack_name)
for _, row in stack_spec_model_df.iterrows():
    print("year", year)
    for cycle, degradation in enumerate(row):
        print("Cycle No:",int(cycles[cycle]))
        print("Degradation Value:", degradation * 100)
        print(" ")
    year = year + 1

In [ ]:
"""
Reading data for a Stack to store it in the project for future use.
Stack Name = S230E CATL (CB310) 3 and 4 hour (S230E 3+ hrs)
"""
stack_name, cycles, stack_spec_model_df = get_data_for_a_stack(total_specifications, start=24, end=47)
print("Stack Name:", stack_name)
year = 0
for _, row in stack_spec_model_df.iterrows():
    print("year", year)
    for cycle, degradation in enumerate(row):
        print("Cycle No:",int(cycles[cycle]))
        print("Degradation Value:", degradation * 100)
        print(" ")
    year = year + 1

<h3>Storing Constant Data for Project</h3>

In [ ]:
def convert_data_type(data_property):
    """
    Convert list of data into a dictionary, where keys
    represent the years and values represent the values
    against that year. 
    """
    output = {
        index: D(value)
            for index, value in enumerate(data_property, start=1)
    }
    return output

In [ ]:
def get_constant_data_for_project(sheet_name, columns, skiprows):
    """
    Read only specific columns from the excel file.
    """
    data = convert_data_to_df(sheet_name, columns=columns, skiprows=skiprows).iloc[0:40]
    return data


In [ ]:
sheet_data = get_constant_data_for_project("data/sample.xlsx", columns="T:X", skiprows=54)

"""
Get required columns from the data frame and convert them into 
desired behaviour using convert_data_type() function
"""

DEGRADATION_CURVE =  convert_data_type(sheet_data["DEGRADATION_CURVE"])
INCREMENTAL_TRANCHE_DC_OUTPUT = convert_data_type(sheet_data["INCREMENTAL_TRANCHE_DC_OUTPUT"])
DEGRADATION_CURVE_TRANCHE_GT_1 = convert_data_type(sheet_data["DEGRADATION_CURVE_TRANCHE_GT_1"])
INCREMENTAL_PROJECT_DC_OUTPUT = convert_data_type(sheet_data["INCREMENTAL_PROJECT_DC_OUTPUT"])
DEGRADATION_WITH_PG_MATRIX = convert_data_type(sheet_data["DEGRADATION_WITH_PG_MATRIX"])

<h3>Implementing calculations and formulas from an excel file</h3>

In [ ]:
class Tranche:
    """Calculate PG Table Tranche formulas for the project."""

    def __init__(self, tranche_info):
        """Constructor"""
        self.tranche_info = tranche_info
        self.years = [year for year in range(1, 41)]
        self.tranche_degradation_with_pg_matrix_prev = (
            DEGRADATION_CURVE
            if self.tranche_info == 1
            else DEGRADATION_CURVE_TRANCHE_GT_1
        )

    @property
    def incremental_tranche_dc_output(self):
        """
        Incremental Tranche DC  Output (MWhDC Out)
        =IF(E98<2, 0, IFERROR(I53*(Augmentation!L10/Augmentation!$J10),0))
        """
        return INCREMENTAL_TRANCHE_DC_OUTPUT

    @property
    def aggregate_tranche_dc_output(self):
        """
        Aggregate Tranche DC  Output (MWhDC Out)
        =SUM(Hn:H$n)
        """
        # n is a natural number
        incremental_tranche_dc_output = self.incremental_tranche_dc_output
        result = {1: 0, 2: incremental_tranche_dc_output[2]}
        for year in self.years[2:]:
            result[year] = sum(
                [incremental_tranche_dc_output[prev_year] for prev_year in result]
                + [incremental_tranche_dc_output[year]]
            )

        return result

    @property
    def tranche_years(self):
        """
        Total Years for a project
        """
        return self.years

    @property
    def degradation_with_pg_matrix(self):
        """
        A constant value throughout the project
        """
        return DEGRADATION_WITH_PG_MATRIX

    @property
    def pg_matrix_hard_input_soh_delta(self):
        """
        Deg w/ PG Matrix vs HARD INPUT SOH Delta
        """
        degradation_with_pg_matrix = self.degradation_with_pg_matrix
        result = {}
        for year in self.years:
            result[year] = abs(
                round(degradation_with_pg_matrix[year], 5)
                - round(D(self.tranche_degradation_with_pg_matrix_prev[year]), 5)
            )
        return result


In [ ]:
class PgTableProject:
    """PG sheet project table formulas"""

    tranche_1 = None
    tranche_2 = None

    def __init__(self):
        self.years = [year for year in range(1, 41)]
        self.initialize_tranches()

    def initialize_tranches(self):
        for tranche in [tranche for tranche in range(1, 3)]:
            setattr(self, f"tranche_{tranche}", Tranche(tranche_info=tranche),)

    @property
    def project_pg_matrix_hard_input_soh_delta(self):
        """Calculate Deg w/ PG Matrix vs HARD INPUT SOH Delta"""
        tranche_deg_pg_matrix = {}
        for tranche in [tranche for tranche in range(1, 3)]:
            tranche_obj = getattr(self, f"tranche_{tranche}")
            tranche_deg_pg_matrix[tranche] = tranche_obj.pg_matrix_hard_input_soh_delta

        result = {}
        for year in self.years:
            result[year] = max(
                [tranche_deg_pg_matrix[tranche][year] for tranche in [tranche for tranche in range(1, 3)]]
            )

        return result

    @property
    def project_cycles(self):
        result = {1: 0}
        for year in self.years[1:]:
            result[year] = result[year - 1] + self.cycles_per_year
        return result


In [ ]:
pg_table = PgTableProject()
hard_input_soh_delta = pg_table.project_pg_matrix_hard_input_soh_delta

<h3>Compare results from Excel sheet and Code</h3>

In [ ]:
output_data = convert_data_to_df("data/sample.xlsx", columns="M", skiprows=50)

output_soh_delta = output_data["Deg w/ PG Matrix vs HARD INPUT SOH Delta"]

data = [
    {"Excel": output_soh_delta[year-1], "Python": value} for year, value in hard_input_soh_delta.items()
]
table = tabulate.tabulate(data, tablefmt="html", headers="keys")
table